In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten , Conv2D , Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
train_data_dir = 'data/'

In [3]:
img_width, img_height = 224, 224
batch_size = 32
epochs = 100
num_classes = 5

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 2931 images belonging to 5 classes.
Found 731 images belonging to 5 classes.


In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [6]:
for layer in base_model.layers:
    layer.trainable = False

In [7]:
x = base_model.output
x = Conv2D(1024 , activation='relu' , kernel_size=(3 , 3))(x)
x = Flatten()(x)
x = Dense(2048, activation='relu')(x)
x = Dense(512 , activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128 , activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(32 , activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [8]:
model = Model(inputs=base_model.input, outputs=predictions)

In [9]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/100
91/91 [==============================] - 105s 1s/step - loss: 1.1716 - accuracy: 0.6350 - val_loss: 0.8728 - val_accuracy: 0.6960
Epoch 2/100
91/91 [==============================] - 44s 481ms/step - loss: 0.8933 - accuracy: 0.6927 - val_loss: 0.8082 - val_accuracy: 0.7060
Epoch 3/100
91/91 [==============================] - 44s 483ms/step - loss: 0.8356 - accuracy: 0.7061 - val_loss: 0.7920 - val_accuracy: 0.7045
Epoch 4/100
91/91 [==============================] - 44s 478ms/step - loss: 0.8094 - accuracy: 0.7137 - val_loss: 0.7756 - val_accuracy: 0.7116
Epoch 5/100
91/91 [==============================] - 45s 493ms/step - loss: 0.7808 - accuracy: 0.7182 - val_loss: 0.7849 - val_accuracy: 0.7116
Epoch 6/100
91/91 [==============================] - 44s 475ms/step - loss: 0.7691 - accuracy: 0.7196 - val_loss: 0.7706 - val_accuracy: 0.7131
Epoch 7/100
91/91 [==============================] - 44s 479ms/step - loss: 0.7578 - accuracy: 0.7223 - val_loss: 0.7142 - val_accuracy: 0

In [11]:
model.save('model.h5')

In [25]:
from pickle import dump
with open('classes.h5' , 'wb') as f:
    dump(train_generator.class_indices , f)